<a href="https://colab.research.google.com/github/Aldo05art/DataCleansingStudents/blob/main/DataMining_Students.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re


In [ ]:
from google.colab import files
uploaded = files.upload()  # jalankan, file kamuu disini yaa
# uploaded adalah dict; ambil nama file pertama
file_name = next(iter(uploaded))
df = pd.read_csv(file_name)

Saving students.csv to students (1).csv


In [ ]:
print("Shape:", df.shape)
display(df.head(8))
print("\nInfo:")
display(df.info())
print("\nMissing values per column:")
print(df.isnull().sum())
print("\nDuplicate rows (exact match):", df.duplicated().sum())

Shape: (200, 7)


,StudentID,Name,Age,Email,Department,GPA,GraduationYear
0,3336,David Palmer,19,sean43@hotmail.com,Mathematics,3.16,2026
1,8774,Andrew Roach,23,vbecker@harvey.com,Chemistry,3.75,2027
2,1396,Jonathan Gonzalez,22,hollydavis@gmail.com,Physics,2.95,2027
3,6716,Kenneth Morrow,24,ganderson@wheeler-atkins.info,Physics,3.55,2029
4,8830,Kaitlyn Martinez,18,hayesdiane@gmail.com,Chemistry,2.29,2025
5,5305,Tiffany Wolf,23,qanderson@taylor.com,Mathematics,3.30,2029
6,5048,James Reyes,20,drodriguez@nguyen-cooper.info,Chemistry,2.44,2029
7,5986,Samantha Sellers,20,michelle27@hubbard-webster.com,Mathematics,3.44,2025



Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   StudentID       200 non-null    int64  
 1   Name            200 non-null    object 
 2   Age             200 non-null    int64  
 3   Email           200 non-null    object 
 4   Department      200 non-null    object 
 5   GPA             200 non-null    float64
 6   GraduationYear  200 non-null    int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 11.1+ KB


None


Missing values per column:
StudentID         0
Name              0
Age               0
Email             0
Department        0
GPA               0
GraduationYear    0
dtype: int64

Duplicate rows (exact match): 0


In [ ]:
df.columns = [c.strip().replace(" ", "_") for c in df.columns]
# jika mau pakai nama tertentu:
# df = df.rename(columns={"StudentID":"student_id", "Name":"name", ...})
df.columns = [c.lower() for c in df.columns]  # semua huruf kecil

In [ ]:
# Trim strings
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].astype(str).str.strip()


In [ ]:
# Sesuaikan nama kolom jika berbeda; asumsi kolom: studentid, name, age, email, department, gpa, graduationyear
# Jika nama kolom berbeda, ganti pada baris berikut.
col_map = {
    "studentid": "student_id",
    "student_id": "student_id",
    "name": "name",
    "age": "age",
    "email": "email",
    "department": "department",
    "gpa": "gpa",
    "graduationyear": "graduation_year",
    "graduation_year": "graduation_year"
}

In [ ]:
# rename if present
for k, v in col_map.items():
    if k in df.columns and v not in df.columns:
        df = df.rename(columns={k: v})

In [ ]:
expected_cols = ["student_id", "name", "age", "email", "department", "gpa", "graduation_year"]
present = [c for c in expected_cols if c in df.columns]
print("Found columns:", present)


Found columns: ['student_id', 'name', 'age', 'email', 'department', 'gpa', 'graduation_year']


In [ ]:
# Convert numeric-like cols
if "age" in df.columns:
    df["age"] = pd.to_numeric(df["age"], errors="coerce")
if "gpa" in df.columns:
    df["gpa"] = pd.to_numeric(df["gpa"], errors="coerce")
if "graduation_year" in df.columns:
    df["graduation_year"] = pd.to_numeric(df["graduation_year"], errors="coerce")


In [ ]:
# Student ID: treat as string (some IDs start with zero)
if "student_id" in df.columns:
    df["student_id"] = df["student_id"].astype(str).str.strip()

In [ ]:
# normalkan text kolom
if "department" in df.columns:
    # remove extra spaces and title-case department names
    df["department"] = df["department"].astype(str).str.replace(r"\s+", " ", regex=True).str.strip().str.title()
if "name" in df.columns:
    df["name"] = df["name"].astype(str).str.replace(r"\s+", " ", regex=True).str.strip().str.title()
if "email" in df.columns:
    df["email"] = df["email"].astype(str).str.strip().str.lower()

In [ ]:
# 5. Mendeteksi apakah terdapat paksaan menciptakan nilai-nilai baru yang hilang
print("\nMissing after type coercion:")
print(df[["age","gpa","graduation_year"]].isnull().sum())



Missing after type coercion:
age                0
gpa                0
graduation_year    0
dtype: int64


In [ ]:
# 6.  Duplicate handling
# Periksa duplikat berdasarkan seluruh baris
dupe_rows = df[df.duplicated(keep=False)]
print("\nExact duplicate rows found:", dupe_rows.shape[0])
# Check duplicate student_id
if "student_id" in df.columns:
    dup_ids = df[df.duplicated(subset=["student_id"], keep=False)]
    print("Rows with duplicate student_id:", dup_ids.shape[0])


Exact duplicate rows found: 0
Rows with duplicate student_id: 0


In [ ]:
# Strategy: drop exact duplicated rows (keep first)
if dupe_rows.shape[0] > 0:
    df = df.drop_duplicates(keep="first").reset_index(drop=True)


In [ ]:
# 7. Validasi format email dan dicoba perbaikan ringan
email_pattern = r'^[\w\.-]+@[\w\.-]+\.\w+$'
if "email" in df.columns:
    # remove spaces and some common artifacts
    df["email"] = df["email"].str.replace(r"\s+", "", regex=True)
    # fix common " at " / "[at]" -> "@"
    df["email"] = df["email"].str.replace(r"\s+at\s+", "@", regex=True)
    df["email"] = df["email"].str.replace(r"\[at\]", "@", regex=True)
    df["email_valid"] = df["email"].str.match(email_pattern)
    invalid_emails = df[~df["email_valid"]]
    print("\nInvalid email count:", invalid_emails.shape[0])
    if invalid_emails.shape[0] > 0:
        display(invalid_emails[["student_id","name","email"]].head(8))


Invalid email count: 0


In [ ]:
# 8. Pemeriksaan apakah make sense & tanda outlier
# a) GPA range: valid range 0.0 - 4.0
if "gpa" in df.columns:
    df["gpa_valid_range"] = df["gpa"].between(0.0, 4.0)
    print("\nGPA values outside 0-4:", (~df["gpa_valid_range"]).sum())



GPA values outside 0-4: 0


In [ ]:
# b) Kelayakan usia (menyesuaikan batasan sesuai kebutuhan)
min_age, max_age = 16, 60
if "age" in df.columns:
    df["age_plausible"] = df["age"].between(min_age, max_age)
    print("Ages outside plausible range ({}-{}):".format(min_age, max_age), (~df["age_plausible"]).sum())


Ages outside plausible range (16-60): 0


In [ ]:
# c) Kelayakan tahun kelulusan
current_year = pd.Timestamp.now().year
min_grad, max_grad = current_year - 10, current_year + 6
if "graduation_year" in df.columns:
    df["graduation_year_plausible"] = df["graduation_year"].between(min_grad, max_grad)
    print("Graduation years outside plausible range ({}-{}):".format(min_grad, max_grad), (~df["graduation_year_plausible"]).sum())


Graduation years outside plausible range (2015-2031): 0


In [ ]:
# d) Deteksi outlier IQR untuk kolom numerik (usia, IPK)
def iqr_outliers(series):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return (series < lower) | (series > upper)

if "gpa" in df.columns:
    df["gpa_outlier"] = iqr_outliers(df["gpa"].dropna())
    print("GPA outliers (IQR):", df["gpa_outlier"].sum())
if "age" in df.columns:
    df["age_outlier"] = iqr_outliers(df["age"].dropna())
    print("Age outliers (IQR):", df["age_outlier"].sum())

GPA outliers (IQR): 0
Age outliers (IQR): 0


In [ ]:
# 9 Tentukan remediasi (contoh):
# - Untuk beberapa email tidak valid: koreksi manual disarankan (periksa invalid_emails).
# - Untuk jumlah kecil nilai numerik yang hilang: imputasi dengan median.
# - Untuk outlier: pertahankan, batasi, atau tetapkan ke NaN dan dokumentasikan.

# Contoh remediasi otomatis (gunakan dengan hati-hati; lebih baik mendokumentasikan dan meninjau):
# Imputasi usia/IPK yang hilang dengan median jika jumlah yang hilang kecil
for col in ["age", "gpa", "graduation_year"]:
    if col in df.columns:
        missing_count = df[col].isnull().sum()
        if 0 < missing_count <= 10:  # small number -> impute with median
            median_val = df[col].median(skipna=True)
            df[col] = df[col].fillna(median_val)
            print(f"Imputed {missing_count} missing values in {col} with median {median_val:.2f}")


In [ ]:
# Untuk nilai IPK di luar 0-4, atur ke NaN (agar dapat diperiksa)
if "gpa" in df.columns:
    bad_gpa = (~df["gpa_valid_range"])
    if bad_gpa.sum() > 0:
        df.loc[bad_gpa, "gpa"] = np.nan
        print("Set out-of-range GPA to NaN for manual review.")

In [ ]:
# 10 Final checks and summary
print("\nFinal missing counts:")
print(df.isnull().sum())
print("\nFinal shape:", df.shape)



Final missing counts:
student_id                   0
name                         0
age                          0
email                        0
department                   0
gpa                          0
graduation_year              0
email_valid                  0
gpa_valid_range              0
age_plausible                0
graduation_year_plausible    0
gpa_outlier                  0
age_outlier                  0
dtype: int64

Final shape: (200, 13)


In [ ]:
# Simpan kolom log pembersihan sederhana yang mengumpulkan flag untuk peninjauan manual
def make_notes(row):
    notes = []
    if "email_valid" in row and not row["email_valid"]:
        notes.append("invalid_email")
    if "gpa_valid_range" in row and not row["gpa_valid_range"]:
        notes.append("gpa_out_of_range")
    if "gpa_outlier" in row and row["gpa_outlier"]:
        notes.append("gpa_outlier")
    if "age_plausible" in row and not row["age_plausible"]:
        notes.append("age_implausible")
    if "age_outlier" in row and row["age_outlier"]:
        notes.append("age_outlier")
    if "graduation_year_plausible" in row and not row["graduation_year_plausible"]:
        notes.append("grad_year_implausible")
    return ";".join(notes) if notes else ""

df["clean_notes"] = df.apply(make_notes, axis=1)

In [ ]:
# melihat rows that need manual review
to_review = df[df["clean_notes"] != ""]
print("\nRows flagged for review:", to_review.shape[0])
if to_review.shape[0] > 0:
    display(to_review.head(10))


Rows flagged for review: 0


In [ ]:
# 11. Save cleaned dataset to a new CSV
out_name = "students_cleaned.csv"
df.to_csv(out_name, index=False)
print("\nCleaned file saved as:", out_name)
# Provide download link in Colab
files.download(out_name)


Cleaned file saved as: students_cleaned.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>